In [1]:
import pandas as pd
import numpy as np
import nltk
import sys
import pickle as pk
import string as st
import warnings
warnings.filterwarnings("ignore")


from bs4 import BeautifulSoup
import unicodedata
import re

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

from nltk.corpus import stopwords

from nltk.corpus import wordnet
from nltk import pos_tag
from nltk import ne_chunk

from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from nltk.probability import FreqDist

In [2]:
df = pd.read_csv('/Users/anusharajan/Mtweets_republic_process.csv')
df.head()

,Tweet,Date,Link,Channel,Unnamed: 4
0,"Priyanka Chopra, Nick Jonas Post Strong Statem...",2020-05-30 23:30:00+00:00,https://twitter.com/republic/status/1266874530...,republic,NaN
1,Trump lauds US' 'bold return to space' after N...,2020-05-30 23:29:29+00:00,https://twitter.com/republic/status/1266874403...,republic,NaN
2,Varun Dhawan Celebrates Niece Niyara Dhawan’s ...,2020-05-30 22:37:00+00:00,https://twitter.com/republic/status/1266861192...,republic,NaN
3,NASA-SpaceX launch provides Trump moment to re...,2020-05-30 22:14:13+00:00,https://twitter.com/republic/status/1266855459...,republic,NaN
4,"Biden lauds NASA-SpaceX Crew Dragon launch, sa...",2020-05-30 22:06:27+00:00,https://twitter.com/republic/status/1266853505...,republic,NaN


In [3]:
#Drop the unnecessary columns in dataset
df = df.drop(['Date', 'Link', 'Unnamed: 4'], axis = 1)
df.head()

,Tweet,Channel
0,"Priyanka Chopra, Nick Jonas Post Strong Statem...",republic
1,Trump lauds US' 'bold return to space' after N...,republic
2,Varun Dhawan Celebrates Niece Niyara Dhawan’s ...,republic
3,NASA-SpaceX launch provides Trump moment to re...,republic
4,"Biden lauds NASA-SpaceX Crew Dragon launch, sa...",republic


In [4]:
#Text Preprocessing - Convert to lower case

df['Tweet'] = [token.lower() for token in df['Tweet']]
df.head(5)

,Tweet,Channel
0,"priyanka chopra, nick jonas post strong statem...",republic
1,trump lauds us' 'bold return to space' after n...,republic
2,varun dhawan celebrates niece niyara dhawan’s ...,republic
3,nasa-spacex launch provides trump moment to re...,republic
4,"biden lauds nasa-spacex crew dragon launch, sa...",republic


In [5]:
#Text Preprocessing - Remove hyperlinks
df['Tweet'] = df['Tweet'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

In [6]:
#Text Preprocessing - Remove punctuations
# Remove all punctuations from the text
import string as st
def remove_punct(text):
    return ("".join([ch for ch in text if ch not in st.punctuation]))
df['removed_punc'] = df['Tweet'].apply(lambda x: remove_punct(x))
df.head()

,Tweet,Channel,removed_punc
0,"priyanka chopra, nick jonas post strong statem...",republic,priyanka chopra nick jonas post strong stateme...
1,trump lauds us' 'bold return to space' after n...,republic,trump lauds us bold return to space after nasa...
2,varun dhawan celebrates niece niyara dhawan’s ...,republic,varun dhawan celebrates niece niyara dhawan’s ...
3,nasa-spacex launch provides trump moment to re...,republic,nasaspacex launch provides trump moment to rel...
4,"biden lauds nasa-spacex crew dragon launch, sa...",republic,biden lauds nasaspacex crew dragon launch says...


In [8]:
df['tokenize'] = df['removed_punc'].apply(lambda x:word_tokenize(x))
df.head(5)

,Tweet,Channel,removed_punc,tokenize
0,"priyanka chopra, nick jonas post strong statem...",republic,priyanka chopra nick jonas post strong stateme...,"[priyanka, chopra, nick, jonas, post, strong, ..."
1,trump lauds us' 'bold return to space' after n...,republic,trump lauds us bold return to space after nasa...,"[trump, lauds, us, bold, return, to, space, af..."
2,varun dhawan celebrates niece niyara dhawan’s ...,republic,varun dhawan celebrates niece niyara dhawan’s ...,"[varun, dhawan, celebrates, niece, niyara, dha..."
3,nasa-spacex launch provides trump moment to re...,republic,nasaspacex launch provides trump moment to rel...,"[nasaspacex, launch, provides, trump, moment, ..."
4,"biden lauds nasa-spacex crew dragon launch, sa...",republic,biden lauds nasaspacex crew dragon launch says...,"[biden, lauds, nasaspacex, crew, dragon, launc..."


In [9]:
#remove stopwords 
def remove_stopwords(text):
    return [word for word in text if word not in nltk.corpus.stopwords.words('english')]

In [16]:
df['clean_tokens'] = df['tokenize'].apply(lambda x : remove_stopwords(x))

In [18]:
df.head(10)

,Tweet,Channel,removed_punc,tokenize,clean_tokens
0,"priyanka chopra, nick jonas post strong statem...",republic,priyanka chopra nick jonas post strong stateme...,"[priyanka, chopra, nick, jonas, post, strong, ...","[priyanka, chopra, nick, jonas, post, strong, ..."
1,trump lauds us' 'bold return to space' after n...,republic,trump lauds us bold return to space after nasa...,"[trump, lauds, us, bold, return, to, space, af...","[trump, lauds, us, bold, return, space, nasasp..."
2,varun dhawan celebrates niece niyara dhawan’s ...,republic,varun dhawan celebrates niece niyara dhawan’s ...,"[varun, dhawan, celebrates, niece, niyara, dha...","[varun, dhawan, celebrates, niece, niyara, dha..."
3,nasa-spacex launch provides trump moment to re...,republic,nasaspacex launch provides trump moment to rel...,"[nasaspacex, launch, provides, trump, moment, ...","[nasaspacex, launch, provides, trump, moment, ..."
4,"biden lauds nasa-spacex crew dragon launch, sa...",republic,biden lauds nasaspacex crew dragon launch says...,"[biden, lauds, nasaspacex, crew, dragon, launc...","[biden, lauds, nasaspacex, crew, dragon, launc..."
5,'because i'm just a human who's allowed to...'...,republic,because im just a human whos allowed to zaira ...,"[because, im, just, a, human, whos, allowed, t...","[im, human, whos, allowed, zaira, wasim, retur..."
6,spacex launch with two nasa astronauts is succ...,republic,spacex launch with two nasa astronauts is succ...,"[spacex, launch, with, two, nasa, astronauts, ...","[spacex, launch, two, nasa, astronauts, succes..."
7,bridenstine was 'praying' for us astronauts on...,republic,bridenstine was praying for us astronauts on c...,"[bridenstine, was, praying, for, us, astronaut...","[bridenstine, praying, us, astronauts, crew, d..."
8,sonu sood meets maharashtra governor; briefs a...,republic,sonu sood meets maharashtra governor briefs ab...,"[sonu, sood, meets, maharashtra, governor, bri...","[sonu, sood, meets, maharashtra, governor, bri..."
9,nasa-spacex mission: watch crew dragon lift-of...,republic,nasaspacex mission watch crew dragon liftoff f...,"[nasaspacex, mission, watch, crew, dragon, lif...","[nasaspacex, mission, watch, crew, dragon, lif..."


In [26]:
relevant = df[df['Tweet'].str.contains('tablighi', na = False)]

In [27]:
relevant.head()

,Tweet,Channel,removed_punc,tokenize,clean_tokens
1641,delhi police to file 20 chargesheets against a...,republic,delhi police to file 20 chargesheets against a...,"[delhi, police, to, file, 20, chargesheets, ag...","[delhi, police, file, 20, chargesheets, around..."
2471,tablighi jamaat members aided covid-19 spread ...,republic,tablighi jamaat members aided covid19 spread i...,"[tablighi, jamaat, members, aided, covid19, sp...","[tablighi, jamaat, members, aided, covid19, sp..."
6283,#indiaagainstlobby | why should action not be ...,republic,indiaagainstlobby why should action not be ta...,"[indiaagainstlobby, why, should, action, not, ...","[indiaagainstlobby, action, taken, tablighi, j..."
6286,#indiaagainstlobby | why should action not be ...,republic,indiaagainstlobby why should action not be ta...,"[indiaagainstlobby, why, should, action, not, ...","[indiaagainstlobby, action, taken, tablighi, j..."
6641,#staystrongindia | if we'd have locked down a ...,republic,staystrongindia if wed have locked down a mon...,"[staystrongindia, if, wed, have, locked, down,...","[staystrongindia, wed, locked, month, earlier,..."


In [37]:
relevant.to_csv('name.csv')

In [6]:
sdf = pd.read_csv('/Users/anusharajan/Mtweets_scroll.csv')
sdf.head()

,Tweet,Date,Link,Publication,Hashtag
0,There should be a global inquiry into the Covi...,2020-05-30 23:00:00+00:00,https://twitter.com/scroll_in/status/126686698...,scroll_in,NaN
1,This translation of two of Chandrasekhara Kamb...,2020-05-30 22:30:00+00:00,https://twitter.com/scroll_in/status/126685943...,scroll_in,NaN
2,From Williams on sale to Mercedes exit rumours...,2020-05-30 22:00:00+00:00,https://twitter.com/scroll_in/status/126685188...,scroll_in,NaN
3,The dos and don’ts of sex in the time of socia...,2020-05-30 21:30:00+00:00,https://twitter.com/scroll_in/status/126684433...,scroll_in,NaN
4,"For India’s $200-billion IT industry, what doe...",2020-05-30 21:00:00+00:00,https://twitter.com/scroll_in/status/126683678...,scroll_in,NaN


In [7]:
#Text Preprocessing - Convert to lower case

sdf['Tweet'] = [token.lower() for token in sdf['Tweet']]
sdf.head(5)

,Tweet,Date,Link,Publication,Hashtag
0,there should be a global inquiry into the covi...,2020-05-30 23:00:00+00:00,https://twitter.com/scroll_in/status/126686698...,scroll_in,NaN
1,this translation of two of chandrasekhara kamb...,2020-05-30 22:30:00+00:00,https://twitter.com/scroll_in/status/126685943...,scroll_in,NaN
2,from williams on sale to mercedes exit rumours...,2020-05-30 22:00:00+00:00,https://twitter.com/scroll_in/status/126685188...,scroll_in,NaN
3,the dos and don’ts of sex in the time of socia...,2020-05-30 21:30:00+00:00,https://twitter.com/scroll_in/status/126684433...,scroll_in,NaN
4,"for india’s $200-billion it industry, what doe...",2020-05-30 21:00:00+00:00,https://twitter.com/scroll_in/status/126683678...,scroll_in,NaN


In [8]:
#Text Preprocessing - Remove hyperlinks
sdf['Tweet'] = sdf['Tweet'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

In [9]:
relevant_scroll = sdf[sdf['Tweet'].str.contains('tablighi', na = False)]

In [10]:
relevant_scroll.to_csv('relevant_scroll.csv')

In [2]:
#import dataset
df = pd.read_csv('/Users/anusharajan/Mtweets_ANI.csv')

In [10]:
#add column names
df.columns = ['Tweet', 'Date', 'Link', 'Publication', 'Hashtag']

In [11]:
df.head()

,Tweet,Date,Link,Publication,Hashtag
0,Madhya Pradesh: Medical staff performed 'aarti...,2020-05-30 23:39:24+00:00,https://twitter.com/ANI/status/126687689689930...,ANI,['COVID19']
1,Death toll in Brazil due to #coronavirus hits ...,2020-05-30 22:53:18+00:00,https://twitter.com/ANI/status/126686529667911...,ANI,['coronavirus']
2,We had placed an order for 60 machines which i...,2020-05-30 22:35:43+00:00,https://twitter.com/ANI/status/126686087129002...,ANI,['Locustsattack']
3,Rajasthan: Locusts swarms were seen in Chhatta...,2020-05-30 22:34:20+00:00,https://twitter.com/ANI/status/126686052302437...,ANI,NaN
4,I am thrilled to announce that SpaceX Dragon C...,2020-05-30 22:17:53+00:00,https://twitter.com/ANI/status/126685638263392...,ANI,NaN


In [12]:
#make tweet column lowercase
df['Tweet'] = [token.lower() for token in df['Tweet']]

In [14]:
#remove hyperlinks
df['Tweet'] = df['Tweet'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

In [15]:
#parse the tweets column for tablighi
relevant = df[df['Tweet'].str.contains('tablighi', na = False)]

In [16]:
#tokenize the tweets
relevant['tokenize'] = relevant['Tweet'].apply(lambda x:word_tokenize(x))

In [17]:
#remove stopwords
def remove_stopwords(text):
    return [word for word in text if word not in nltk.corpus.stopwords.words('english')]

relevant['clean_tokens'] = relevant['tokenize'].apply(lambda x : remove_stopwords(x))

In [ ]:
#export to csv
relevant.to_csv('relevant_ANI.csv')